In [1]:
import json
import pickle

import networkx as nx
import requests

In [11]:
with open('pickle/star-trek_only_relevant_properties_dict.pickle', 'rb') as f:
    only_relevant_properties_dict = pickle.load(f)

with open('pickle/star-trek_only_relevant_types_dict.pickle', 'rb') as f:
    only_relevant_types_dict = pickle.load(f)

with open('pickle/star-trek_all_instances_and_types_dict.pickle', 'rb') as f:
    all_instances_and_types_dict = pickle.load(f)

with open('pickle/star-trek_hierarchy_graph.pickle', 'rb') as f:
    hierarchy = pickle.load(f)

In [12]:
relevant_instances_dict = {} # all relevant information for instances of relevant types

In [13]:
def get_direct_superclasses(c):
    all_neighbors = hierarchy.neighbors(c)
    direct_superclasses = list(filter(lambda x: hierarchy[c][x]['type'] == 'is_subclass_of', all_neighbors))
    return direct_superclasses

def get_superclasses(c):
    direct_superclasses = get_direct_superclasses(c)
    if not direct_superclasses:
        superclasses = [[c]]
    else:
        superclasses = [[c] + spclsss for c2 in direct_superclasses for spclsss in get_superclasses(c2)]
    return superclasses

def is_subclass_of(c1, c2):
    return any(map(lambda x: c2 in x, get_superclasses(c1)))

def get_closest_class(c, cs):
    cs = list(cs.keys())
    lengths = []
    for c2 in cs:
        try:
            # WARNING: if there is more than one shortest path, then it returns only one of them
            length = nx.shortest_path_length(hierarchy, c, c2)
        except nx.exception.NetworkXNoPath as e:
            length = 999
        lengths.append(length)
    min_length = min(lengths)
    closest = cs[lengths.index(min_length)]
    count_min = lengths.count(min_length)
    return closest

def get_closest_relevant_class(c):
    return get_closest_class(c, only_relevant_types_dict)

def is_subclass_of_any_relevant_class(c):
    return any(map(lambda x: is_subclass_of(c, x), only_relevant_types_dict))

def get_relevant_type(ide):
    relevant_type = None
    if ide in all_instances_and_types_dict:
        types = all_instances_and_types_dict[ide]
        if len(types) == 1:
            t = types.pop(); types.add(t)
            if is_subclass_of_any_relevant_class(t):
                relevant_type = get_closest_relevant_class(t)
        else:
            relevant_candidates = set()
            for t in types:
                if is_subclass_of_any_relevant_class(t):
                    relevant_type = get_closest_relevant_class(t)
                    relevant_candidates.add(relevant_type)
            if relevant_candidates:
                # WARNING: if there is more than one relevant_candidates, then it returns only one of them
                relevant_type = relevant_candidates.pop(); relevant_candidates.add(relevant_type)
    return relevant_type

In [14]:
# def get_name(ide):
#     created = False
#     base_url = 'https://www.wikidata.org/w/api.php'
#     payload = {'action': 'wbgetentities', 'ids': ide, 'languages': 'en', 'format': 'json'}
#     r = requests.get(base_url, params=payload)
#     response = r.json()
#     d = response['entities'][ide]
#     # ide = d['id']
#     if 'en' in d['labels']:
#         name = d['labels']['en']['value']
#     else:
#         name = ''
#     return name

In [15]:
# def get_values(prop):
#     values_r = set()
#     values_a = set()
#     for prop_inst in d['claims'][prop]:
#         prop_type = prop_inst['mainsnak']['datavalue']['type']
#         if prop_type == 'wikibase-entityid':
#             value = 'Q' + str(prop_inst['mainsnak']['datavalue']['value']['numeric-id'])
#             if value in all_instances_and_types_dict:
#                 values_r.add(value)
#             else:
#                 values_a.add(value)
#         elif prop_type == 'string':
#             value = str(prop_inst['mainsnak']['datavalue']['value'])
#             values_a.add(value)
#         elif prop_type == 'time':
#             value = str(prop_inst['mainsnak']['datavalue']['value']['time'][1:11])
#             values_a.add(value)
#         elif prop_type == 'monolingualtext':
#             value = str(prop_inst['mainsnak']['datavalue']['value']['text'])
#             values_a.add(value)
#         else:
#             # WARNING: Unknown property type
#             raise Exception('Unknown property type:', ide, prop, type_prop)
#     return (values_r, values_a)

In [20]:
def get_value(prop_inst):
    try:
        main_info = prop_inst['mainsnak'] # for properties themselves
    except KeyError as e:
        main_info = prop_inst # for qualifiers
    prop_type = main_info['datavalue']['type']
    if prop_type == 'wikibase-entityid':
        value = 'Q' + str(main_info['datavalue']['value']['numeric-id'])
    elif prop_type == 'string':
        value = str(main_info['datavalue']['value'])
    elif prop_type == 'time':
        value = str(main_info['datavalue']['value']['time'][1:11])
    elif prop_type == 'monolingualtext':
        value = str(main_info['datavalue']['value']['text'])
    else:
        # WARNING: Unknown property type
        raise Exception('Unknown property type')
    return value

In [21]:
# %%time
total = 0
count = 0
error = 0
relevant_instances_dict = {} # all relevant information for instances of relevant types
with open('data/wikidata-20150907-star_trek.json') as f:
    for line in f:
#         try:
        if not line.startswith('[') and not line.startswith(']'):
            total += 1
            cleaned = line.strip()[:-1]
            d = json.loads(cleaned) # keys: ['type', 'labels', 'descriptions', 'claims', 'aliases', 'sitelinks', 'id']
            ide = d['id']
            typ = get_relevant_type(ide)
            if typ:
                if 'en' in d['labels']:
                    name = d['labels']['en']['value']
                else:
                    name = ''
                if 'enwiki' in d['sitelinks']:
                    wikilink = d['sitelinks']['enwiki']['title']
                else:
                    wikilink = ''
                if 'en' in d['descriptions']:
                    description = d['descriptions']['en']['value']
                else:
                    description = ''
                info = {
                    'id': ide,
                    'type': typ,
                    'name': name,
                    'wikilink': wikilink,
                    'description': description,
                    'properties': {},
                }
                for prop in d['claims']:
                    if prop in only_relevant_properties_dict and prop != 'P31' and prop != 'P279':
                        values = []
                        for prop_inst in d['claims'][prop]:
                            p_value = get_value(prop_inst)
                            qualifiers = {}
                            if 'qualifiers' in prop_inst:
                                for q in prop_inst['qualifiers']:
                                    if q in only_relevant_properties_dict:
                                        q_values = []
                                        for q_inst in prop_inst['qualifiers'][q]:
                                            q_value = get_value(q_inst)
                                            q_values.append(q_value)
                                        qualifiers[q] = q_values
                            values.append({'value': p_value, 'qualifiers': qualifiers})
                        info['properties'][prop] = values
                relevant_instances_dict[ide] = info
                count += 1
#         except Exception as e:
#             error += 1
print('Total:', total)
print('Count:', count)
print('Error:', error)

Total: 1628
Count: 915
Error: 0


In [22]:
len(relevant_instances_dict)

915

In [23]:
relevant_instances_dict

{'Q10134': {'description': 'constructed language',
  'id': 'Q10134',
  'name': 'Klingon language',
  'properties': {'P1080': [{'qualifiers': {}, 'value': 'Q18043309'}],
   'P373': [{'qualifiers': {}, 'value': 'Klingon language'}],
   'P646': [{'qualifiers': {}, 'value': '/m/049ph'}],
   'P910': [{'qualifiers': {}, 'value': 'Q8574920'}]},
  'type': 'Q33215',
  'wikilink': 'Klingon language'},
 'Q1016685': {'description': 'Star Trek: Voyager episode',
  'id': 'Q1016685',
  'name': 'The Cloud',
  'properties': {'P155': [{'qualifiers': {}, 'value': 'Q661255'}],
   'P156': [{'qualifiers': {}, 'value': 'Q5422700'}],
   'P179': [{'qualifiers': {}, 'value': 'Q156329'}],
   'P345': [{'qualifiers': {}, 'value': 'tt0708978'}],
   'P361': [{'qualifiers': {}, 'value': 'Q3465791'}],
   'P364': [{'qualifiers': {}, 'value': 'Q1860'}],
   'P495': [{'qualifiers': {}, 'value': 'Q30'}],
   'P646': [{'qualifiers': {}, 'value': '/m/09d915'}]},
  'type': 'Q1983062',
  'wikilink': 'The Cloud (Star Trek: Voyag

In [12]:
{"type":"item","id":"Q17738","labels":{"es":{"language":"es","value":"Star Wars: Episode IV - A New Hope"},"af":{"language":"af","value":"Star Wars Episode IV: A New Hope"},"ar":{"language":"ar","value":"\u062d\u0631\u0628 \u0627\u0644\u0646\u062c\u0648\u0645 \u0627\u0644\u062c\u0632\u0621 \u0627\u0644\u0631\u0627\u0628\u0639: \u0623\u0645\u0644 \u062c\u062f\u064a\u062f"},"bg":{"language":"bg","value":"\u041c\u0435\u0436\u0434\u0443\u0437\u0432\u0435\u0437\u0434\u043d\u0438 \u0432\u043e\u0439\u043d\u0438: \u0415\u043f\u0438\u0437\u043e\u0434 IV - \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0435\u0436\u0434\u0430"},"bn":{"language":"bn","value":"\u09b8\u09cd\u099f\u09be\u09b0 \u0993\u09af\u09bc\u09be\u09b0\u09cd\u09b8 - \u09a6\u09bf \u09a8\u09bf\u0989 \u09b9\u09cb\u09aa"},"ca":{"language":"ca","value":"Star Wars episodi IV: Una nova esperan\u00e7a"},"cs":{"language":"cs","value":"Star Wars: Epizoda IV - Nov\u00e1 nad\u011bje"},"da":{"language":"da","value":"Star Wars Episode IV: Et nyt h\u00e5b"},"de":{"language":"de","value":"Krieg der Sterne"},"el":{"language":"el","value":"\u039f \u03a0\u03cc\u03bb\u03b5\u03bc\u03bf\u03c2 \u03c4\u03c9\u03bd \u0386\u03c3\u03c4\u03c1\u03c9\u03bd: \u0395\u03c0\u03b5\u03b9\u03c3\u03cc\u03b4\u03b9\u03bf 4 - \u039c\u03b9\u03b1 \u039d\u03ad\u03b1 \u0395\u03bb\u03c0\u03af\u03b4\u03b1"},"en":{"language":"en","value":"Star Wars Episode IV: A New Hope"},"eo":{"language":"eo","value":"Star Wars Episode IV: A New Hope"},"eu":{"language":"eu","value":"Star Wars IV. Atala: Itxaropen Berria"},"fa":{"language":"fa","value":"\u062c\u0646\u06af \u0633\u062a\u0627\u0631\u06af\u0627\u0646 \u0627\u067e\u06cc\u0632\u0648\u062f \u0686\u0647\u0627\u0631\u0645: \u0627\u0645\u06cc\u062f\u06cc \u062a\u0627\u0632\u0647"},"fi":{"language":"fi","value":"T\u00e4htien sota"},"fo":{"language":"fo","value":"Star Wars Episode IV: Ein n\u00fdggj v\u00f3n"},"fr":{"language":"fr","value":"Star Wars, \u00e9pisode IV : Un nouvel espoir"},"ga":{"language":"ga","value":"Star Wars Episode IV: A New Hope"},"gl":{"language":"gl","value":"Star Wars Episode IV: A New Hope"},"he":{"language":"he","value":"\u05de\u05dc\u05d7\u05de\u05ea \u05d4\u05db\u05d5\u05db\u05d1\u05d9\u05dd - \u05e4\u05e8\u05e7 4: \u05ea\u05e7\u05d5\u05d5\u05d4 \u05d7\u05d3\u05e9\u05d4"},"hi":{"language":"hi","value":"\u0938\u094d\u091f\u093e\u0930 \u0935\u0949\u0930\u094d\u0938 \u090f\u092a\u093f\u0938\u094b\u0921 IV: \u0905 \u0928\u094d\u092f\u0942 \u0939\u094b\u092a"},"hr":{"language":"hr","value":"Zvjezdani ratovi IV: Nova nada"},"hu":{"language":"hu","value":"Csillagok h\u00e1bor\u00faja IV: \u00daj rem\u00e9ny"},"is":{"language":"is","value":"Sj\u00f6rnustr\u00ed\u00f0: N\u00fd von"},"it":{"language":"it","value":"Guerre stellari"},"ja":{"language":"ja","value":"\u30b9\u30bf\u30fc\u30fb\u30a6\u30a9\u30fc\u30ba \u30a8\u30d4\u30bd\u30fc\u30c94\/\u65b0\u305f\u306a\u308b\u5e0c\u671b"},"ka":{"language":"ka","value":"\u10d5\u10d0\u10e0\u10e1\u10d9\u10d5\u10da\u10d0\u10d5\u10e3\u10e0\u10d8 \u10dd\u10db\u10d4\u10d1\u10d8, \u10d4\u10de\u10d8\u10d6\u10dd\u10d3\u10d8 IV: \u10d0\u10ee\u10d0\u10da\u10d8 \u10d8\u10db\u10d4\u10d3\u10d8"},"ko":{"language":"ko","value":"\uc2a4\ud0c0 \uc6cc\uc988 \uc5d0\ud53c\uc18c\ub4dc 4: \uc0c8\ub85c\uc6b4 \ud76c\ub9dd"},"la":{"language":"la","value":"Star Wars Episode IV: A New Hope"},"lb":{"language":"lb","value":"Star Wars Episode IV: A New Hope"},"lt":{"language":"lt","value":"\u017dvaig\u017ed\u017ei\u0173 karai: epizodas IV - nauja viltis"},"lv":{"language":"lv","value":"Zvaig\u017e\u0146u kari: Jaun\u0101 cer\u012bba"},"ms":{"language":"ms","value":"Star Wars Episode IV: A New Hope"},"nl":{"language":"nl","value":"Star Wars: Episode IV: A New Hope"},"pl":{"language":"pl","value":"Gwiezdne wojny: cz\u0119\u015b\u0107 IV - Nowa nadzieja"},"pt":{"language":"pt","value":"Star Wars Epis\u00f3dio IV: Uma Nova Esperan\u00e7a"},"ro":{"language":"ro","value":"R\u0103zboiul stelelor - Episodul IV: O nou\u0103 speran\u021b\u0103"},"ru":{"language":"ru","value":"\u0417\u0432\u0451\u0437\u0434\u043d\u044b\u0435 \u0432\u043e\u0439\u043d\u044b. \u042d\u043f\u0438\u0437\u043e\u0434 IV: \u041d\u043e\u0432\u0430\u044f \u043d\u0430\u0434\u0435\u0436\u0434\u0430"},"sa":{"language":"sa","value":"\u0938\u094d\u091f\u093e\u0930\u094d \u0935\u093e\u0930\u094d\u0938\u094d:\u0905 \u0928\u0941\u092f\u094d\u092f\u094d \u0939\u094b\u092a\u094d"},"sh":{"language":"sh","value":"Star Wars Episode IV: A New Hope"},"sk":{"language":"sk","value":"Hviezdne vojny"},"sr":{"language":"sr","value":"\u0417\u0432\u0435\u0437\u0434\u0430\u043d\u0438 \u0440\u0430\u0442\u043e\u0432\u0438 \u2014 \u0435\u043f\u0438\u0437\u043e\u0434\u0430 IV: \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0430"},"sv":{"language":"sv","value":"Stj\u00e4rnornas krig"},"th":{"language":"th","value":"\u0e2a\u0e15\u0e32\u0e23\u0e4c \u0e27\u0e2d\u0e23\u0e4c\u0e2a \u0e40\u0e2d\u0e1e\u0e1e\u0e34\u0e42\u0e0b\u0e14 4: \u0e04\u0e27\u0e32\u0e21\u0e2b\u0e27\u0e31\u0e07\u0e43\u0e2b\u0e21\u0e48"},"tr":{"language":"tr","value":"Y\u0131ld\u0131z Sava\u015flar\u0131: B\u00f6l\u00fcm IV - Yeni Bir Umut"},"uk":{"language":"uk","value":"\u0417\u043e\u0440\u044f\u043d\u0456 \u0432\u0456\u0439\u043d\u0438. \u0415\u043f\u0456\u0437\u043e\u0434 IV. \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0456\u044f"},"zh":{"language":"zh","value":"\u661f\u969b\u5927\u6230\u56db\u90e8\u66f2\uff1a\u66d9\u5149\u4e4d\u73fe"},"hy":{"language":"hy","value":"\u0531\u057d\u057f\u0572\u0561\u0575\u056b\u0576 \u057a\u0561\u057f\u0565\u0580\u0561\u0566\u0574\u0576\u0565\u0580. \u0537\u057a\u056b\u0566\u0578\u0564 IV. \u0546\u0578\u0580 \u0570\u0578\u0582\u0575\u057d"},"wa":{"language":"wa","value":"Li Guere des Sitoeles Episode 4 : On novea Espwer"},"mk":{"language":"mk","value":"\u0412\u043e\u0458\u043d\u0430 \u043d\u0430 \u0455\u0432\u0435\u0437\u0434\u0438\u0442\u0435: \u0415\u043f\u0438\u0437\u043e\u0434\u0430 IV: \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0435\u0436"},"be-tarask":{"language":"be-tarask","value":"\u0417\u043e\u0440\u043d\u044b\u044f \u0432\u043e\u0439\u043d\u044b. \u042d\u043f\u0456\u0437\u043e\u0434 IV: \u041d\u043e\u0432\u0430\u044f \u043d\u0430\u0434\u0437\u0435\u044f"},"nb":{"language":"nb","value":"Star Wars Episode IV: Et nytt h\u00e5p"},"mr":{"language":"mr","value":"\u0938\u094d\u091f\u093e\u0930 \u0935\u0949\u0930\u094d\u0938 \u092d\u093e\u0917 \u096a: \u0905 \u0928\u094d\u092f\u0942 \u0939\u094b\u092a"},"nn":{"language":"nn","value":"Star Wars Episode IV: A New Hope"},"id":{"language":"id","value":"Star Wars Episode IV: A New Hope"},"sco":{"language":"sco","value":"Star Wars Episode IV: A New Hope"},"br":{"language":"br","value":"Star Wars Lodenn 4: Ur Spi Nevez"},"kk":{"language":"kk","value":"\u0416\u04b1\u043b\u0434\u044b\u0437\u0434\u0430\u0440 \u0448\u0430\u0439\u049b\u0430\u0441\u044b. IV \u044d\u043f\u0438\u0437\u043e\u0434: \u0416\u0430\u04a3\u0430 \u04af\u043c\u0456\u0442"},"be":{"language":"be","value":"\u0417\u043e\u0440\u043d\u044b\u044f \u0432\u043e\u0439\u043d\u044b. \u042d\u043f\u0456\u0437\u043e\u0434 IV: \u041d\u043e\u0432\u0430\u044f \u043d\u0430\u0434\u0437\u0435\u044f"},"bs":{"language":"bs","value":"Zvjezdani ratovi IV: Nova nada"}},"descriptions":{"nl":{"language":"nl","value":"Amerikaanse sciencefictionfilm uit 1977"},"de":{"language":"de","value":"Spielfilm der Star-Wars-Reihe"},"es":{"language":"es","value":"pel\u00edcula de space opera de 1977"},"it":{"language":"it","value":"film del 1977 scritto e diretto da George Lucas"},"hu":{"language":"hu","value":"1977-es amerikai film"},"en":{"language":"en","value":"1977 American epic space opera film"},"fr":{"language":"fr","value":"film am\u00e9ricain de la saga Star Wars sorti en 1977"},"fa":{"language":"fa","value":"\u0641\u06cc\u0644\u0645\u06cc \u0627\u0632 \u062c\u0631\u062c \u0644\u0648\u06a9\u0627\u0633"},"gl":{"language":"gl","value":"Filme de 1977 dirixido por George Lucas"},"cs":{"language":"cs","value":"\u010ctvrt\u00e1 epizoda Star Wars z roku 1977"}},"aliases":{"de":[{"language":"de","value":"Star Wars"},{"language":"de","value":"Star Wars: Episode IV \u2013 A New Hope"}],"es":[{"language":"es","value":"Star Wars: episodio IV - Una nueva esperanza"},{"language":"es","value":"La guerra de las galaxias"}],"hu":[{"language":"hu","value":"Csillagok h\u00e1bor\u00faja"},{"language":"hu","value":"Csillagok h\u00e1bor\u00faja IV: Egy \u00faj rem\u00e9ny"},{"language":"hu","value":"Star Wars IV: \u00daj rem\u00e9ny"},{"language":"hu","value":"Star Wars IV: Egy \u00faj rem\u00e9ny"}],"en":[{"language":"en","value":"Star Wars"},{"language":"en","value":"Star Wars IV"},{"language":"en","value":"Star Wars 4"}],"it":[{"language":"it","value":"Star Wars: Episodio IV - Una nuova speranza"}],"fr":[{"language":"fr","value":"La Guerre des \u00e9toiles"},{"language":"fr","value":"Un nouvel espoir"},{"language":"fr","value":"La Guerre des \u00e9toiles, IV"},{"language":"fr","value":"Star Wars IV"},{"language":"fr","value":"Star Wars 4"}],"cs":[{"language":"cs","value":"Nov\u00e1 nad\u011bje"},{"language":"cs","value":"Star Wars: Epizoda IV"}]},"claims":{"P1237":[{"mainsnak":{"snaktype":"value","property":"P1237","datavalue":{"value":"starwars4","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$edbc4b75-4889-3d21-92be-93e05ac02d24","rank":"normal"}],"P1265":[{"mainsnak":{"snaktype":"value","property":"P1265","datavalue":{"value":"25801","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$4e08f6b4-47bf-2f9e-ff6e-57f446ac93b2","rank":"normal"}],"P272":[{"mainsnak":{"snaktype":"value","property":"P272","datavalue":{"value":{"entity-type":"item","numeric-id":242446},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$78CC4072-1B39-484C-A876-25D6D5A386DD","rank":"normal"}],"P57":[{"mainsnak":{"snaktype":"value","property":"P57","datavalue":{"value":{"entity-type":"item","numeric-id":38222},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$D87BEE7E-5117-421E-85BF-8702804669EF","rank":"normal"}],"P136":[{"mainsnak":{"snaktype":"value","property":"P136","datavalue":{"value":{"entity-type":"item","numeric-id":468478},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$25710235-793A-4DFC-8A01-769640B93631","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P136","datavalue":{"value":{"entity-type":"item","numeric-id":471839},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$4D62331E-27FA-4048-AD6B-42A6FFB7D6A8","rank":"normal","references":[{"hash":"7eb64cf9621d34c54fd4bd040ed4b61a88c4a1a0","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":328},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P136","datavalue":{"value":{"entity-type":"item","numeric-id":188473},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$3D05CD75-562F-49E8-A693-6D992FA90E36","rank":"normal","references":[{"hash":"7eb64cf9621d34c54fd4bd040ed4b61a88c4a1a0","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":328},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P136","datavalue":{"value":{"entity-type":"item","numeric-id":319221},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$D6384416-587B-436C-A433-7F62AC8D003A","rank":"normal"}],"P179":[{"mainsnak":{"snaktype":"value","property":"P179","datavalue":{"value":{"entity-type":"item","numeric-id":462},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P155":[{"snaktype":"value","property":"P155","hash":"f7e52a7a2e88205ce48a5d7577cc8b9cc94bc289","datavalue":{"value":{"entity-type":"item","numeric-id":42051},"type":"wikibase-entityid"},"datatype":"wikibase-item"}],"P156":[{"snaktype":"value","property":"P156","hash":"ce259bf3b190c4c795cd57cdc74aa307a025a2fe","datavalue":{"value":{"entity-type":"item","numeric-id":181795},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P155","P156"],"id":"q17738$C6964B20-0031-4017-9D2A-C5D9244F0903","rank":"normal"}],"P31":[{"mainsnak":{"snaktype":"value","property":"P31","datavalue":{"value":{"entity-type":"item","numeric-id":11424},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$844338C2-2A22-404A-AA4B-6B41B5CFB8B6","rank":"normal","references":[{"hash":"d6e3ab4045fb3f3feea77895bc6b27e663fc878a","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":206855},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P345":[{"mainsnak":{"snaktype":"value","property":"P345","datavalue":{"value":"tt0076759","type":"string"},"datatype":"string"},"type":"statement","id":"q17738$A8D614D1-EE50-45AC-94FD-5F7FAB06806E","rank":"normal","references":[{"hash":"dbe2189e7b0d3953de6f69cc1cc88109a79605a4","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":8449},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P58":[{"mainsnak":{"snaktype":"value","property":"P58","datavalue":{"value":{"entity-type":"item","numeric-id":38222},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$0B07E182-E307-4A21-A3C3-0AB208288421","rank":"normal"}],"P162":[{"mainsnak":{"snaktype":"value","property":"P162","datavalue":{"value":{"entity-type":"item","numeric-id":1494912},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$EAB5068D-D95A-417F-962A-BABEC91A1630","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P162","datavalue":{"value":{"entity-type":"item","numeric-id":683785},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$4A7A9875-26C9-4C67-BF2F-03725F087D72","rank":"normal","references":[{"hash":"50f57a3dbac4708ce4ae4a827c0afac7fcdb4a5c","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":11920},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P344":[{"mainsnak":{"snaktype":"value","property":"P344","datavalue":{"value":{"entity-type":"item","numeric-id":182148},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$CDCB0809-035B-4CAF-86B4-C00F2BA97942","rank":"normal"}],"P161":[{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":81328},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"1bdaafcf423b8902d43ba9c402fef5b2fb1edf5d","datavalue":{"value":{"entity-type":"item","numeric-id":51802},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$6E2B56F0-5C7D-4E03-92BD-597FD05A49FA","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":103894},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"ddbf66bf88c889aef6cb26a694592a9fc2790f8e","datavalue":{"value":{"entity-type":"item","numeric-id":51740},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$EF67CF32-128B-47A7-9561-C66680C4DCB7","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":108941},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"c33f7602a1b3df9ec3bfe4eaee9efbca1428ed81","datavalue":{"value":{"entity-type":"item","numeric-id":51797},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$14302895-6D22-477B-98F1-9D8CEA387E1B","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":210447},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"e0243c947785e580238264935c203c87bec46f7b","datavalue":{"value":{"entity-type":"item","numeric-id":51746},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$0457387C-0C3C-4565-8C8D-FA9CB1AC968D","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":309589},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"963892bfd122ffdaa48a9b3d98d8edd1a30bce92","datavalue":{"value":{"entity-type":"item","numeric-id":51800},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$0061A404-C555-426F-B3BB-1BF324DE638B","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":560552},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"46f38ab62cf1f3ad99dcb73a1590a4095d05c118","datavalue":{"value":{"entity-type":"item","numeric-id":51803},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$C880EAD1-8702-4625-AA79-9A80516ED245","rank":"normal","references":[{"hash":"50f57a3dbac4708ce4ae4a827c0afac7fcdb4a5c","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":11920},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":128379},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"4e1af317a1004a767e8582c1f0234de1e6d2fa02","datavalue":{"value":{"entity-type":"item","numeric-id":12206942},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$B13B3272-7D45-44E0-8362-DF3C55560BE1","rank":"normal","references":[{"hash":"50f57a3dbac4708ce4ae4a827c0afac7fcdb4a5c","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":11920},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":441685},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"9473dba8364062fca60ee933a473d2a6914386bc","datavalue":{"value":{"entity-type":"item","numeric-id":51787},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$5621E8ED-C568-4630-A1EC-6BFBC489D962","rank":"normal","references":[{"hash":"50f57a3dbac4708ce4ae4a827c0afac7fcdb4a5c","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":11920},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":454081},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"43d4f40c3d5bf4acfac0bdd7f7361e53e685d56c","datavalue":{"value":{"entity-type":"item","numeric-id":51788},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$FE5556C9-7443-4E33-B17F-F9DB06F941FF","rank":"normal","references":[{"hash":"50f57a3dbac4708ce4ae4a827c0afac7fcdb4a5c","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":11920},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":1187504},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"a0bb20ec8ab909766c5a9b131d015d8bfee4cdb1","datavalue":{"value":{"entity-type":"item","numeric-id":52310},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$96D9C94E-C6D7-4803-B3A7-EFAA10EECDCA","rank":"normal","references":[{"hash":"50f57a3dbac4708ce4ae4a827c0afac7fcdb4a5c","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":11920},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":367085},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"3392f71e5c88a874d7d8255ec21429b545cf8e35","datavalue":{"value":{"entity-type":"item","numeric-id":2158602},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$1275AC31-F145-4D4F-9DAF-5E7013BF99BD","rank":"normal","references":[{"hash":"50f57a3dbac4708ce4ae4a827c0afac7fcdb4a5c","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":11920},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":325398},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"26a5fee07b7a5b6a8ec4e55fdb538a74c326d31f","datavalue":{"value":{"entity-type":"item","numeric-id":2267163},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$FAF9254D-6ABE-4F30-B05A-A275701B5921","rank":"normal","references":[{"hash":"004ec6fbee857649acdbdbad4f97b2c8571df97b","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":48183},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":512456},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"1a437f7cb5b3d612c6946d416503eba949c93992","datavalue":{"value":{"entity-type":"item","numeric-id":2480769},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"q17738$6250D6F7-D8BF-4283-9AB7-7767D48341E6","rank":"normal","references":[{"hash":"004ec6fbee857649acdbdbad4f97b2c8571df97b","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":48183},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]},{"mainsnak":{"snaktype":"value","property":"P161","datavalue":{"value":{"entity-type":"item","numeric-id":1351454},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P453":[{"snaktype":"value","property":"P453","hash":"d614a32a58d4edb1a0a91c2ebaf4c9ee76eac5d0","datavalue":{"value":{"entity-type":"item","numeric-id":517292},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"qualifiers-order":["P453"],"id":"Q17738$F3CAE55E-145C-4F05-B9F6-E59A8FA4EEB0","rank":"normal","references":[{"hash":"f70116eac7f49194478b3025330bfd8dcffa3c69","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":8447},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P373":[{"mainsnak":{"snaktype":"value","property":"P373","datavalue":{"value":"Star Wars","type":"string"},"datatype":"string"},"type":"statement","id":"q17738$70E4C253-2DCB-4372-89DD-E26AA9844541","rank":"normal"}],"P480":[{"mainsnak":{"snaktype":"value","property":"P480","datavalue":{"value":"712041","type":"string"},"datatype":"string"},"type":"statement","id":"q17738$03B04E1B-4B47-4B11-98ED-16C3781879B0","rank":"normal","references":[{"hash":"dbe2189e7b0d3953de6f69cc1cc88109a79605a4","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":8449},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P364":[{"mainsnak":{"snaktype":"value","property":"P364","datavalue":{"value":{"entity-type":"item","numeric-id":1860},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"q17738$D76A8225-5DF7-4DBF-8D1B-2F2D219F97C3","rank":"normal","references":[{"hash":"7eb64cf9621d34c54fd4bd040ed4b61a88c4a1a0","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":328},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P214":[{"mainsnak":{"snaktype":"value","property":"P214","datavalue":{"value":"183464503","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$3BFC3961-337A-4BF3-AEBF-D3346A9C8557","rank":"normal","references":[{"hash":"004ec6fbee857649acdbdbad4f97b2c8571df97b","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":48183},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P244":[{"mainsnak":{"snaktype":"value","property":"P244","datavalue":{"value":"n78001095","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$E98206E0-C83F-4DE9-8036-07E2EFB2F18F","rank":"normal","references":[{"hash":"004ec6fbee857649acdbdbad4f97b2c8571df97b","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":48183},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P856":[{"mainsnak":{"snaktype":"value","property":"P856","datavalue":{"value":"http:\/\/www.starwars.com\/movies\/episode-iv","type":"string"},"datatype":"url"},"type":"statement","id":"Q17738$22ABE42E-819A-4920-B1BF-4BEE6A6C10C8","rank":"normal","references":[{"hash":"7eb64cf9621d34c54fd4bd040ed4b61a88c4a1a0","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":328},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P910":[{"mainsnak":{"snaktype":"value","property":"P910","datavalue":{"value":{"entity-type":"item","numeric-id":8810557},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$8E55B0DD-AEE4-4960-BDA2-B91FE37727FA","rank":"normal"}],"P646":[{"mainsnak":{"snaktype":"value","property":"P646","datavalue":{"value":"\/m\/0dtfn","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$19A15933-8613-499A-9072-584C0C03CFF1","rank":"normal","references":[{"hash":"af38848ab5d9d9325cffd93a5ec656cc6ca889ed","snaks":{"P248":[{"snaktype":"value","property":"P248","datavalue":{"value":{"entity-type":"item","numeric-id":15241312},"type":"wikibase-entityid"},"datatype":"wikibase-item"}],"P577":[{"snaktype":"value","property":"P577","datavalue":{"value":{"time":"+2013-10-28T00:00:00Z","timezone":0,"before":0,"after":0,"precision":11,"calendarmodel":"http:\/\/www.wikidata.org\/entity\/Q1985727"},"type":"time"},"datatype":"time"}]},"snaks-order":["P248","P577"]}]}],"P577":[{"mainsnak":{"snaktype":"value","property":"P577","datavalue":{"value":{"time":"+1977-05-25T00:00:00Z","timezone":0,"before":0,"after":0,"precision":11,"calendarmodel":"http:\/\/www.wikidata.org\/entity\/Q1985727"},"type":"time"},"datatype":"time"},"type":"statement","id":"Q17738$3486C460-AB91-411A-8FDB-7DF1DD784128","rank":"normal","references":[{"hash":"ee68603cf4fcd172052bbcb77a1ce2df535ac97f","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":766705},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P18":[{"mainsnak":{"snaktype":"value","property":"P18","datavalue":{"value":"Star Wars Logo.svg","type":"string"},"datatype":"commonsMedia"},"type":"statement","id":"Q17738$27CF1803-83E9-412F-AF8F-395C70A6DEAB","rank":"normal","references":[{"hash":"223f43a7229297bd67ffab922174f80c722fff1a","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":10000},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P1258":[{"mainsnak":{"snaktype":"value","property":"P1258","datavalue":{"value":"m\/star_wars","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$8F86A5FA-1D10-4DE1-B0A4-7EA9D695ECDC","rank":"normal","references":[{"hash":"7eb64cf9621d34c54fd4bd040ed4b61a88c4a1a0","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":328},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P905":[{"mainsnak":{"snaktype":"value","property":"P905","datavalue":{"value":"408","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$83ed107f-4aba-89dc-9898-1b403f73502b","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P905","datavalue":{"value":"7324","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$fee55f0e-4a09-ab9a-6757-3d2fca2e659a","rank":"normal"}],"P495":[{"mainsnak":{"snaktype":"value","property":"P495","datavalue":{"value":{"entity-type":"item","numeric-id":30},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$530DFC12-646D-4C86-AE6A-9C9EB08BB932","rank":"normal"}],"P1562":[{"mainsnak":{"snaktype":"value","property":"P1562","datavalue":{"value":"v46636","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$9C9827E0-EF88-46FD-8EEA-20339D436F87","rank":"normal","references":[{"hash":"7eb64cf9621d34c54fd4bd040ed4b61a88c4a1a0","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":328},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P155":[{"mainsnak":{"snaktype":"value","property":"P155","datavalue":{"value":{"entity-type":"item","numeric-id":28787},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$E499D8EC-D323-4ACA-901D-02819F6CFEA0","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P155","datavalue":{"value":{"entity-type":"item","numeric-id":42051},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$505182EF-7A3A-445E-9A42-F967DBF58E30","rank":"normal"}],"P166":[{"mainsnak":{"snaktype":"value","property":"P166","datavalue":{"value":{"entity-type":"item","numeric-id":823422},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","qualifiers":{"P585":[{"snaktype":"value","property":"P585","hash":"3390eddfbc6a9df21b5bb0719fce7a1d8b447dad","datavalue":{"value":{"time":"+1989-01-01T00:00:00Z","timezone":0,"before":0,"after":0,"precision":9,"calendarmodel":"http:\/\/www.wikidata.org\/entity\/Q1985727"},"type":"time"},"datatype":"time"}]},"qualifiers-order":["P585"],"id":"Q17738$E6833706-03C4-49B9-A2CB-44FDDC9EF46A","rank":"normal","references":[{"hash":"7eb64cf9621d34c54fd4bd040ed4b61a88c4a1a0","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":328},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P1657":[{"mainsnak":{"snaktype":"value","property":"P1657","datavalue":{"value":{"entity-type":"item","numeric-id":18665334},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$05db3495-4411-1960-4f86-52d9a965495e","rank":"normal"}],"P915":[{"mainsnak":{"snaktype":"value","property":"P915","datavalue":{"value":{"entity-type":"item","numeric-id":774},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$33949847-20C1-40B5-B56B-E25BE34E2179","rank":"normal"},{"mainsnak":{"snaktype":"value","property":"P915","datavalue":{"value":{"entity-type":"item","numeric-id":99},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$6762F382-3EB3-458E-82C9-FD038E3FFDAD","rank":"normal"}],"P1970":[{"mainsnak":{"snaktype":"value","property":"P1970","datavalue":{"value":"84","type":"string"},"datatype":"string"},"type":"statement","id":"Q17738$28C336DB-23B1-4B1F-A947-64FDEE8A0A82","rank":"normal","references":[{"hash":"223f43a7229297bd67ffab922174f80c722fff1a","snaks":{"P143":[{"snaktype":"value","property":"P143","datavalue":{"value":{"entity-type":"item","numeric-id":10000},"type":"wikibase-entityid"},"datatype":"wikibase-item"}]},"snaks-order":["P143"]}]}],"P156":[{"mainsnak":{"snaktype":"value","property":"P156","datavalue":{"value":{"entity-type":"item","numeric-id":181795},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$cbd6386c-4fe7-50cd-0c19-1ef9685247c1","rank":"normal"}],"P1431":[{"mainsnak":{"snaktype":"value","property":"P1431","datavalue":{"value":{"entity-type":"item","numeric-id":38222},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$BB6DAE27-8B15-4DFE-8DB6-180F71CA2237","rank":"normal"}],"P86":[{"mainsnak":{"snaktype":"value","property":"P86","datavalue":{"value":{"entity-type":"item","numeric-id":131285},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$4D4955B2-26EC-4AEA-9F42-7D74F0F617F1","rank":"normal"}],"P406":[{"mainsnak":{"snaktype":"value","property":"P406","datavalue":{"value":{"entity-type":"item","numeric-id":2715371},"type":"wikibase-entityid"},"datatype":"wikibase-item"},"type":"statement","id":"Q17738$fbb75c84-44e5-0885-1cbd-448c97a7f785","rank":"normal"}]},"sitelinks":{"afwiki":{"site":"afwiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"bgwiki":{"site":"bgwiki","title":"\u041c\u0435\u0436\u0434\u0443\u0437\u0432\u0435\u0437\u0434\u043d\u0438 \u0432\u043e\u0439\u043d\u0438: \u0415\u043f\u0438\u0437\u043e\u0434 IV - \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0435\u0436\u0434\u0430","badges":[]},"bnwiki":{"site":"bnwiki","title":"\u09b8\u09cd\u099f\u09be\u09b0 \u0993\u09af\u09bc\u09be\u09b0\u09cd\u09b8 - \u09a6\u09bf \u09a8\u09bf\u0989 \u09b9\u09cb\u09aa","badges":[]},"dawiki":{"site":"dawiki","title":"Star Wars Episode IV: Et nyt h\u00e5b","badges":[]},"dewiki":{"site":"dewiki","title":"Krieg der Sterne","badges":[]},"elwiki":{"site":"elwiki","title":"\u039f \u03a0\u03cc\u03bb\u03b5\u03bc\u03bf\u03c2 \u03c4\u03c9\u03bd \u0386\u03c3\u03c4\u03c1\u03c9\u03bd: \u0395\u03c0\u03b5\u03b9\u03c3\u03cc\u03b4\u03b9\u03bf 4 - \u039c\u03b9\u03b1 \u039d\u03ad\u03b1 \u0395\u03bb\u03c0\u03af\u03b4\u03b1","badges":[]},"enwiki":{"site":"enwiki","title":"Star Wars (film)","badges":[]},"eowiki":{"site":"eowiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"euwiki":{"site":"euwiki","title":"Star Wars IV. Atala: Itxaropen Berria","badges":[]},"fawiki":{"site":"fawiki","title":"\u062c\u0646\u06af \u0633\u062a\u0627\u0631\u06af\u0627\u0646 (\u0641\u06cc\u0644\u0645)","badges":[]},"fiwiki":{"site":"fiwiki","title":"T\u00e4htien sota (alkuper\u00e4inen elokuva)","badges":[]},"fowiki":{"site":"fowiki","title":"Star Wars Episode IV: Ein n\u00fdggj v\u00f3n","badges":[]},"gawiki":{"site":"gawiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"glwiki":{"site":"glwiki","title":"A guerra das galaxias, episodio IV: Unha esperanza nova","badges":[]},"hewiki":{"site":"hewiki","title":"\u05de\u05dc\u05d7\u05de\u05ea \u05d4\u05db\u05d5\u05db\u05d1\u05d9\u05dd - \u05e4\u05e8\u05e7 4: \u05ea\u05e7\u05d5\u05d5\u05d4 \u05d7\u05d3\u05e9\u05d4","badges":[]},"hiwiki":{"site":"hiwiki","title":"\u0938\u094d\u091f\u093e\u0930 \u0935\u0949\u0930\u094d\u0938 \u090f\u092a\u093f\u0938\u094b\u0921 IV: \u0905 \u0928\u094d\u092f\u0942 \u0939\u094b\u092a","badges":[]},"hrwiki":{"site":"hrwiki","title":"Zvjezdani ratovi IV: Nova nada","badges":[]},"huwiki":{"site":"huwiki","title":"Csillagok h\u00e1bor\u00faja IV: \u00daj rem\u00e9ny","badges":[]},"iswiki":{"site":"iswiki","title":"Star Wars: A New Hope","badges":[]},"itwiki":{"site":"itwiki","title":"Guerre stellari (film)","badges":[]},"jawiki":{"site":"jawiki","title":"\u30b9\u30bf\u30fc\u30fb\u30a6\u30a9\u30fc\u30ba \u30a8\u30d4\u30bd\u30fc\u30c94\/\u65b0\u305f\u306a\u308b\u5e0c\u671b","badges":[]},"kawiki":{"site":"kawiki","title":"\u10d5\u10d0\u10e0\u10e1\u10d9\u10d5\u10da\u10d0\u10d5\u10e3\u10e0\u10d8 \u10dd\u10db\u10d4\u10d1\u10d8 (\u10e4\u10d8\u10da\u10db\u10d8)","badges":[]},"kowiki":{"site":"kowiki","title":"\uc2a4\ud0c0 \uc6cc\uc988 \uc5d0\ud53c\uc18c\ub4dc 4: \uc0c8\ub85c\uc6b4 \ud76c\ub9dd","badges":[]},"lawiki":{"site":"lawiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"lbwiki":{"site":"lbwiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"ltwiki":{"site":"ltwiki","title":"\u017dvaig\u017ed\u017ei\u0173 karai: epizodas IV \u2013 nauja viltis","badges":[]},"lvwiki":{"site":"lvwiki","title":"Zvaig\u017e\u0146u kari: Jaun\u0101 cer\u012bba","badges":[]},"mswiki":{"site":"mswiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"nlwiki":{"site":"nlwiki","title":"Star Wars: Episode IV: A New Hope","badges":[]},"nowiki":{"site":"nowiki","title":"Star Wars Episode IV: Et nytt h\u00e5p","badges":[]},"plwiki":{"site":"plwiki","title":"Gwiezdne wojny: cz\u0119\u015b\u0107 IV \u2013 Nowa nadzieja","badges":[]},"ptwiki":{"site":"ptwiki","title":"Star Wars Epis\u00f3dio IV: Uma Nova Esperan\u00e7a","badges":[]},"rowiki":{"site":"rowiki","title":"R\u0103zboiul stelelor - Episodul IV: O nou\u0103 speran\u021b\u0103","badges":[]},"ruwiki":{"site":"ruwiki","title":"\u0417\u0432\u0451\u0437\u0434\u043d\u044b\u0435 \u0432\u043e\u0439\u043d\u044b. \u042d\u043f\u0438\u0437\u043e\u0434 IV: \u041d\u043e\u0432\u0430\u044f \u043d\u0430\u0434\u0435\u0436\u0434\u0430","badges":[]},"sawiki":{"site":"sawiki","title":"\u0938\u094d\u091f\u093e\u0930\u094d \u0935\u093e\u0930\u094d\u0938\u094d:\u0905 \u0928\u0941\u092f\u094d\u092f\u094d \u0939\u094b\u092a\u094d","badges":[]},"shwiki":{"site":"shwiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"simplewiki":{"site":"simplewiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"skwiki":{"site":"skwiki","title":"Hviezdne vojny (1977)","badges":[]},"srwiki":{"site":"srwiki","title":"\u0417\u0432\u0435\u0437\u0434\u0430\u043d\u0438 \u0440\u0430\u0442\u043e\u0432\u0438 \u2014 \u0435\u043f\u0438\u0437\u043e\u0434\u0430 IV: \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0430","badges":[]},"svwiki":{"site":"svwiki","title":"Stj\u00e4rnornas krig (film)","badges":[]},"thwiki":{"site":"thwiki","title":"\u0e2a\u0e15\u0e32\u0e23\u0e4c \u0e27\u0e2d\u0e23\u0e4c\u0e2a \u0e40\u0e2d\u0e1e\u0e1e\u0e34\u0e42\u0e0b\u0e14 4: \u0e04\u0e27\u0e32\u0e21\u0e2b\u0e27\u0e31\u0e07\u0e43\u0e2b\u0e21\u0e48","badges":[]},"trwiki":{"site":"trwiki","title":"Y\u0131ld\u0131z Sava\u015flar\u0131: B\u00f6l\u00fcm IV - Yeni Bir Umut","badges":[]},"zhwiki":{"site":"zhwiki","title":"\u661f\u969b\u5927\u6230\u56db\u90e8\u66f2\uff1a\u66d9\u5149\u4e4d\u73fe","badges":[]},"wawiki":{"site":"wawiki","title":"Li Guere des Stoeles, Epiz\u00f4de 4 : On novea Espwer","badges":[]},"nnwiki":{"site":"nnwiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"be_x_oldwiki":{"site":"be_x_oldwiki","title":"\u0417\u043e\u0440\u043d\u044b\u044f \u0432\u043e\u0439\u043d\u044b. \u042d\u043f\u0456\u0437\u043e\u0434 IV: \u041d\u043e\u0432\u0430\u044f \u043d\u0430\u0434\u0437\u0435\u044f","badges":[]},"mrwiki":{"site":"mrwiki","title":"\u0938\u094d\u091f\u093e\u0930 \u0935\u0949\u0930\u094d\u0938 \u092d\u093e\u0917 \u096a: \u0905 \u0928\u094d\u092f\u0942 \u0939\u094b\u092a","badges":[]},"itwikiquote":{"site":"itwikiquote","title":"Guerre stellari (film 1977)","badges":[]},"kawikiquote":{"site":"kawikiquote","title":"\u10d5\u10d0\u10e0\u10e1\u10d9\u10d5\u10da\u10d0\u10d5\u10e3\u10e0\u10d8 \u10dd\u10db\u10d4\u10d1\u10d8, \u10d4\u10de\u10d8\u10d6\u10dd\u10d3\u10d8 IV: \u10d0\u10ee\u10d0\u10da\u10d8 \u10d8\u10db\u10d4\u10d3\u10d8","badges":[]},"enwikiquote":{"site":"enwikiquote","title":"Star Wars Episode IV: A New Hope","badges":[]},"hrwikiquote":{"site":"hrwikiquote","title":"Zvjezdani ratovi IV: Nova nada","badges":[]},"ruwikiquote":{"site":"ruwikiquote","title":"\u0417\u0432\u0451\u0437\u0434\u043d\u044b\u0435 \u0432\u043e\u0439\u043d\u044b. \u042d\u043f\u0438\u0437\u043e\u0434 IV. \u041d\u043e\u0432\u0430\u044f \u043d\u0430\u0434\u0435\u0436\u0434\u0430","badges":[]},"dewikiquote":{"site":"dewikiquote","title":"Star Wars: Episode IV \u2013 Eine neue Hoffnung","badges":[]},"frwikiquote":{"site":"frwikiquote","title":"Star Wars, \u00e9pisode IV : Un nouvel espoir (film)","badges":[]},"huwikiquote":{"site":"huwikiquote","title":"Csillagok h\u00e1bor\u00faja IV: Egy \u00faj rem\u00e9ny","badges":[]},"ukwikiquote":{"site":"ukwikiquote","title":"\u0417\u043e\u0440\u044f\u043d\u0456 \u0432\u0456\u0439\u043d\u0438. \u0415\u043f\u0456\u0437\u043e\u0434 IV. \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0456\u044f","badges":[]},"idwiki":{"site":"idwiki","title":"Star Wars Episode IV: A New Hope","badges":[]},"cawiki":{"site":"cawiki","title":"Star Wars episodi IV: Una nova esperan\u00e7a","badges":["Q17437796"]},"eswiki":{"site":"eswiki","title":"Star Wars: Episode IV - A New Hope","badges":["Q17437796"]},"arwiki":{"site":"arwiki","title":"\u062d\u0631\u0628 \u0627\u0644\u0646\u062c\u0648\u0645 (\u0641\u064a\u0644\u0645)","badges":[]},"brwiki":{"site":"brwiki","title":"Star Wars Lodenn 4: Ur Spi Nevez","badges":[]},"cswiki":{"site":"cswiki","title":"Star Wars: Epizoda IV \u2013 Nov\u00e1 nad\u011bje","badges":[]},"kkwiki":{"site":"kkwiki","title":"\u0416\u04b1\u043b\u0434\u044b\u0437\u0434\u0430\u0440 \u0448\u0430\u0439\u049b\u0430\u0441\u044b. IV \u044d\u043f\u0438\u0437\u043e\u0434: \u0416\u0430\u04a3\u0430 \u04af\u043c\u0456\u0442","badges":[]},"bewiki":{"site":"bewiki","title":"\u0417\u043e\u0440\u043d\u044b\u044f \u0432\u043e\u0439\u043d\u044b. \u042d\u043f\u0456\u0437\u043e\u0434 IV: \u041d\u043e\u0432\u0430\u044f \u043d\u0430\u0434\u0437\u0435\u044f","badges":[]},"scowiki":{"site":"scowiki","title":"Star Wars (film)","badges":[]},"frwiki":{"site":"frwiki","title":"Star Wars, \u00e9pisode IV : Un nouvel espoir","badges":["Q17437798"]},"bswiki":{"site":"bswiki","title":"Ratovi zvijezda IV: Nova nada","badges":[]},"ukwiki":{"site":"ukwiki","title":"\u0417\u043e\u0440\u044f\u043d\u0456 \u0432\u0456\u0439\u043d\u0438. \u0415\u043f\u0456\u0437\u043e\u0434 IV. \u041d\u043e\u0432\u0430 \u043d\u0430\u0434\u0456\u044f","badges":[]},"hywiki":{"site":"hywiki","title":"\u0531\u057d\u057f\u0572\u0561\u0575\u056b\u0576 \u057a\u0561\u057f\u0565\u0580\u0561\u0566\u0574\u0576\u0565\u0580: \u0537\u057a\u056b\u0566\u0578\u0564 IV: \u0546\u0578\u0580 \u0570\u0578\u0582\u0575\u057d","badges":[]}}}

{'aliases': {'cs': [{'language': 'cs', 'value': 'Nová naděje'},
   {'language': 'cs', 'value': 'Star Wars: Epizoda IV'}],
  'de': [{'language': 'de', 'value': 'Star Wars'},
   {'language': 'de', 'value': 'Star Wars: Episode IV – A New Hope'}],
  'en': [{'language': 'en', 'value': 'Star Wars'},
   {'language': 'en', 'value': 'Star Wars IV'},
   {'language': 'en', 'value': 'Star Wars 4'}],
  'es': [{'language': 'es',
    'value': 'Star Wars: episodio IV - Una nueva esperanza'},
   {'language': 'es', 'value': 'La guerra de las galaxias'}],
  'fr': [{'language': 'fr', 'value': 'La Guerre des étoiles'},
   {'language': 'fr', 'value': 'Un nouvel espoir'},
   {'language': 'fr', 'value': 'La Guerre des étoiles, IV'},
   {'language': 'fr', 'value': 'Star Wars IV'},
   {'language': 'fr', 'value': 'Star Wars 4'}],
  'hu': [{'language': 'hu', 'value': 'Csillagok háborúja'},
   {'language': 'hu', 'value': 'Csillagok háborúja IV: Egy új remény'},
   {'language': 'hu', 'value': 'Star Wars IV: Új remé

In [ ]:
# Exception: ('Property cannot be relationship and attribute:', 'Q7460439', 'P179')

In [7]:
with open('pickle/star-trek_relevant_instances_dict.pickle', 'wb') as f:
    pickle.dump(relevant_instances_dict, f)

In [8]:
len(relevant_instances_dict)

910

In [36]:
# Check:
# 1 for all instance, no relationship and attribute with same name
# 2 for all relationship, the values are ids contained in relevant_instances_dict
#    2.1 if length of the set of values is equal to 1, then remove relationship
# 3 for all attribute, the length of the set of values is equal to 1
#    3.1 if length of the set of values is equal to 1, then remove attribute
#    3.2 reassign
aux = dict(relevant_instances_dict)
for ide in aux:
    # 1
    relationships = set(aux[ide]['relationships'])
    attributes = set(aux[ide]['attributes'])
    assert(not relationships.intersection(attributes))
#     print(0,relationships.intersection(attributes))
#     if relationships.intersection(attributes):
#         print(1,ide,relationships, attributes)
    # 2
    d = {}
    for rel in aux[ide]['relationships']:
        if aux[ide]['relationships'][rel]:
            d[rel] = aux[ide]['relationships'][rel]
            for val in d[rel]:
                assert(val in aux)
#                 if not val in aux:
#                     print(2,ide,val)
        else:
            print('Relationship', rel, 'empty.')
    aux[ide]['relationships'] = dict(d)
    # 3
    d = {}
    for att in aux[ide]['attributes']:
        if aux[ide]['attributes'][att]:
            d[att] = aux[ide]['attributes'][att]
#             assert(len(aux[ide]['attributes'][att]) == 1)
            if not len(aux[ide]['attributes'][att]) == 1:
                print(3,ide,att,d[att])
                break
            d[att] = d[att].pop()
        else:
            print('Attribute', att, 'empty.')

Attribute P400 empty.
3 Q7600975 P136 {'space simulator', 'action game'}
Attribute P364 empty.
Attribute P155 empty.
Attribute P361 empty.
Attribute P577 empty.
Attribute P854 empty.
Attribute P275 empty.
Attribute P433 empty.
Attribute P156 empty.
Attribute P180 empty.
Attribute P364 empty.
Attribute P495 empty.
Attribute P345 empty.
Attribute P364 empty.
Attribute P646 empty.
Attribute P495 empty.
Attribute P364 empty.
Attribute P495 empty.
Attribute P345 empty.
Attribute P364 empty.
Attribute P495 empty.
Attribute P345 empty.
Attribute P364 empty.
Attribute P646 empty.
Attribute P495 empty.
Attribute P345 empty.
Attribute P364 empty.
Attribute P646 empty.
Attribute P495 empty.
Attribute P577 empty.
Attribute P364 empty.
Attribute P495 empty.
Attribute P345 empty.
Attribute P364 empty.
Attribute P646 empty.
Attribute P495 empty.
Attribute P921 empty.
Attribute P364 empty.
Attribute P495 empty.
Attribute P345 empty.
Attribute P364 empty.
Attribute P495 empty.
Attribute P345 empty.
Att

AssertionError: 